In [ ]:
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install ir_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import os
import requests
import ir_datasets
import random
import numpy as np
import pandas as pd
import pyterrier as pt

In [ ]:
!pip uninstall transformers
!pip install transformers[torch]

Found existing installation: transformers 4.28.1
Uninstalling transformers-4.28.1:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.9/dist-packages/transformers-4.28.1.dist-info/*
    /usr/local/lib/python3.9/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.28.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)


In [ ]:
!pip install git+https://github.com/allenai/ir_datasets.git@crisisfacts 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/allenai/ir_datasets.git (to revision crisisfacts) to /tmp/pip-req-build-g69w5qxu
  Running command git clone --filter=blob:none --quiet https://github.com/allenai/ir_datasets.git /tmp/pip-req-build-g69w5qxu
  Running command git checkout -b crisisfacts --track origin/crisisfacts
  Switched to a new branch 'crisisfacts'
  Branch 'crisisfacts' set up to track remote branch 'crisisfacts' from 'origin'.
  Resolved https://github.com/allenai/ir_datasets.git to commit 6a3e0155dbbd7c1d14bac759660a635aed2288e8
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install python-terrier # install pyTerrier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
credentials = {
    "institution": "University of Foo", # University, Company or Public Agency Name
    "contactname": "Foo Bar", # Your Name
    "email": "foo@bar.edu", # A contact email address
    "institutiontype": "Research" # Either 'Research', 'Industry', or 'Public Sector'
}

home_dir = os.path.expanduser('~')
!mkdir -p ~/.ir_datasets/auth/
with open(home_dir + '/.ir_datasets/auth/crisisfacts.json', 'w') as f:
    json.dump(credentials, f)

In [ ]:
# Event numbers as a list
eventNoList = [
          "001", # Lilac Wildfire 2017
          "002", # Cranston Wildfire 2018
          "003", # Holy Wildfire 2018
          "004", # Hurricane Florence 2018
          "005", # 2018 Maryland Flood
          "006", # Saddleridge Wildfire 2019
          "007", # Hurricane Laura 2020
          "008" # Hurricane Sally 2020
]

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [ ]:
# Gets the list of days for a specified event number, e.g. '001'
def getDaysForEventNo(eventNo):
  # We will download a file containing the day list for an event
  url = "http://trecis.org/CrisisFACTs/CrisisFACTS-"+eventNo+".requests.json"
  # Download the list and parse as JSON
  dayList = requests.get(url).json()
  return dayList

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def dataset_pyterier(data_link):
  if not pt.started():
    pt.init()
  data_link = "irds:"+data_link
  pyTerrierDataset = pt.get_dataset(data_link)
  indexer = pt.IterDictIndexer("None", type=pt.index.IndexingType(3), meta=['docno', 'text'], meta_lengths=[40, 200])
  index = indexer.index(pyTerrierDataset.get_corpus_iter())
  retriever = pt.BatchRetrieve(index, wmodel="DFReeKLIM", metadata=["docno", "text"])
  return retriever

In [ ]:

r=dataset_pyterier('crisisfacts/001/2017-12-07')

crisisfacts/001/2017-12-07 documents: 0it [00:00, ?it/s]

In [ ]:
def preprocess(texts):
    inputs = tokenizer.batch_encode_plus(
        [( text) for text in texts],
        padding='longest',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )
    return inputs

def uni_sen(text):

  # keep track of unique sentences
  unique_sentences = set()
  for sentence in text:
      if sentence not in unique_sentences:
          unique_sentences.add(sentence)

  # join unique sentences back into text
  unique_text = list(unique_sentences)
  return unique_text

def generate_summaries(inputs):
    summaries = []
    for input_ids, attention_mask in zip(inputs['input_ids'], inputs['attention_mask']):
        input_ids = input_ids.unsqueeze(0)
        attention_mask = attention_mask.unsqueeze(0)
        summary_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150, num_beams=2)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        if summary not in summaries:
          summaries.append(summary)
    summaries = uni_sen(summaries)
    return summaries

def summarize_dataframe(df):
    texts = df['text'].tolist()
    inputs = preprocess(texts)
    summaries = generate_summaries(inputs)
    df['summary'] = summaries
    if len(df)>20:
      return df[:20]
    return df

def filter_summaries(query, texts, summaries):
    filtered_summaries = []
    for text, summary in zip(texts, summaries):
        if query.lower() in text.lower():
            filtered_summaries.append(summary)
    return filtered_summaries

In [ ]:
def summary(event,query):
  event_days = getDaysForEventNo(event)
  # Dates for a particular event
  days_list = []
  for d in event_days:
    days_list.append(d["dateString"])
  print(days_list)
  day=str(input())
  event_day_data = 'crisisfacts/'+event+'/'+day
  dataset = ir_datasets.load(event_day_data)
  itemsAsDataFrame = pd.DataFrame(dataset.docs_iter())
  itemQueries = pd.DataFrame(dataset.queries_iter())
  retriever = dataset_pyterier(event_day_data)
  df = pd.DataFrame(retriever.search(query))
  a=summarize_dataframe(df)
  texts = a["text"].to_list()
  summaries = a["summary"].to_list()
  final_summ = filter_summaries(query, texts, summaries)
  print("*********************************************************************************************************************************")
  for t in final_summ:
    print("--> ",t)
    

In [ ]:
summary("001","injuries")

['2017-12-07', '2017-12-08', '2017-12-09', '2017-12-10', '2017-12-11', '2017-12-12', '2017-12-13', '2017-12-14', '2017-12-15']
2017-12-09


crisisfacts/001/2017-12-09 documents: 0it [00:00, ?it/s]

11:13:53.500 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
*********************************************************************************************************************************
-->  #LilacFire @Capt. Heggie @LilacFire #LilacFire #LilacFire #LilacFire #LilacFire #LilacFire #LilacFire #LilacFire #LilacFire #LilacFire #LilacFire #LilacFire #LilacFire #LilacFire #LilacFire #LilacFire #Live
-->  were a separated shoulder and smoke inhalation.
-->  The latest Quirk Injury Law News! https://t.co/H7YOjDb9II Thanks to @MaginnisLawPLLC @vilamaratap #lilacfire #lilacfire #lilacfire #lilacfire #lilacfire #lilacfire #lilacfire #lilacfire #lilacfire #lilacfire #lilacfire #lilacfire #lilacfire #lilacfire #
-->  Horses are much safer today at the Del Mar Fairgrounds. Man